In [2]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *

from pyspark.sql import Row
from pyspark.sql.window import Window
import pandas as pd # our main data management package
import string # used for preprocessing
import re # used for preprocessingq
import numpy as np # used for managing NaNs
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

In [45]:
# 1

# df=spark.read.csv('gs://buck_561/divvy_kag/data.csv',header=True, inferSchema=True)

# df.show(5)

# df.count()

# df.groupby('gender').count().show()

# df.groupby('events').count().show()

# df.groupby('year').count().show()

# df.groupby('usertype').count().show()

# 2

# dx=spark.read.csv('gs://buck_561/divvy_kag/data_raw.csv',header=True,inferSchema=True)

# dx.show(5)

# dx.count()

# dx.groupby('gender').count().show()

# dx.groupby('events').count().show()

# dx.groupby('usertype').count().show()

# dx.groupby('conditions').count().show(50,False)

# dx=dx.withColumn("year",F.substring(F.col("starttime"),0,4))


# dx.groupby('year').count().show()

# 3

In [18]:
d13=spark.read.csv('gs://buck_561/divvy15_21/2013',header=True,inferSchema=True)

d13=d13.withColumn('starttime',F.date_format('starttime',"yyyy-MM-dd hh:mm:ss"))\
.withColumn('stoptime',F.date_format('stoptime',"yyyy-MM-dd hh:mm:ss"))\
.withColumnRenamed('birthday','birthyear')
d13.show(2),d13.count()

+-------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+--------+------+---------+
|trip_id|          starttime|           stoptime|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|usertype|gender|birthyear|
+-------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+--------+------+---------+
|   4118|2013-06-27 12:11:00|2013-06-27 12:16:00|   480|         316|             85|Michigan Ave & Oa...|           28|Larrabee St & Men...|Customer|  null|     null|
|   4275|2013-06-27 02:44:00|2013-06-27 02:45:00|    77|          64|             32|Racine Ave & Cong...|           32|Racine Ave & Cong...|Customer|  null|     null|
+-------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+--------+------+---

(None, 759788)

In [19]:
d14=spark.read.csv('gs://buck_561/divvy15_21/2014',header=True,inferSchema=True)
d14.show(2),d14.count()

+-------+---------------+-------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|trip_id|      starttime|     stoptime|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+-------+---------------+-------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|2355134|6/30/2014 23:57|7/1/2014 0:07|  2006|         604|            131|Lincoln Ave & Bel...|          303|Broadway & Cornel...|Subscriber|  Male|     1988|
|2355133|6/30/2014 23:56|7/1/2014 0:00|  2217|         263|            282|Halsted St & Maxw...|           22|  May St & Taylor St|Subscriber|  Male|     1992|
+-------+---------------+-------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
only showing top 2 rows



(None, 2454634)

In [20]:
d15=spark.read.csv('gs://buck_561/divvy15_21/2015',header=True,inferSchema=True)
d15.show(2),d15.count()

+-------+---------------+-------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|trip_id|      starttime|     stoptime|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+-------+---------------+-------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|5943500|6/30/2015 23:59|7/1/2015 0:10|  1712|         626|            332|Halsted St & Dive...|          327|Sheffield Ave & W...|Subscriber|  Male|     1993|
|5943499|6/30/2015 23:59|7/1/2015 0:23|  4209|        1428|            490|Troy St & Elston Ave|          227|Southport Ave & W...|  Customer|  null|     null|
+-------+---------------+-------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
only showing top 2 rows



(None, 3183439)

In [21]:
d16=spark.read.csv('gs://buck_561/divvy15_21/2016',header=True,inferSchema=True)
d16.show(2),d16.count()

+--------+------------------+------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
| trip_id|         starttime|          stoptime|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+--------+------------------+------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|12150160|9/30/2016 23:59:58|10/1/2016 00:04:03|  4959|         245|             69|Damen Ave & Pierc...|           17|Wood St & Divisio...|Subscriber|  Male|     1988|
|12150159|9/30/2016 23:59:58|10/1/2016 00:04:09|  2589|         251|            383|Ashland Ave & Har...|          320|Loomis St & Lexin...|Subscriber|Female|     1990|
+--------+------------------+------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+----

(None, 3595383)

In [22]:
d17=spark.read.csv('gs://buck_561/divvy15_21/2017',header=True,inferSchema=True)
d17=d17.withColumnRenamed('start_time','starttime')\
.withColumnRenamed('end_time','stoptime')

d17.show(2),d17.count()

+--------+------------------+-----------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
| trip_id|         starttime|         stoptime|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+--------+------------------+-----------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|14853213|6/30/2017 23:59:51|7/1/2017 00:13:57|   893|         846|            107|Desplaines St & J...|           56|Desplaines St & K...|Subscriber|  Male|     1975|
|14853212|6/30/2017 23:59:28|7/1/2017 00:07:10|  1909|         462|            165| Clark St & Grace St|          234|Clark St & Montro...|Subscriber|Female|     1968|
+--------+------------------+-----------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---

(None, 3829014)

In [23]:
d18=spark.read.csv('gs://buck_561/divvy15_21/2018',header=True,inferSchema=True)

d18=d18.withColumnRenamed('start_time','starttime')\
.withColumnRenamed('end_time','stoptime')

d18.show(2),d18.count()

+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
| trip_id|          starttime|           stoptime|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|18000527|2018-04-01 00:04:44|2018-04-01 00:13:03|  3819|       499.0|             22|  May St & Taylor St|          171|May St & Cullerto...|Subscriber|  Male|     1994|
|18000528|2018-04-01 00:06:42|2018-04-01 00:27:07|  5000|     1,225.0|            157|Lake Shore Dr & W...|          190|Southport Ave & W...|Subscriber|  Male|     1965|
+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+---

(None, 3603082)

In [24]:
d19=spark.read.csv('gs://buck_561/divvy15_21/2019',header=True,inferSchema=True)

d19=d19.withColumnRenamed('01 - Rental Details Rental ID','trip_id')\
.withColumnRenamed('01 - Rental Details Local Start Time','starttime')\
.withColumnRenamed('01 - Rental Details Local End Time','stoptime')\
.withColumnRenamed('01 - Rental Details Bike ID','bikeid')\
.withColumnRenamed('01 - Rental Details Duration In Seconds Uncapped','tripduration')\
.withColumnRenamed('03 - Rental Start Station ID','from_station_id')\
.withColumnRenamed('03 - Rental Start Station Name','from_station_name')\
.withColumnRenamed('02 - Rental End Station ID','to_station_id')\
.withColumnRenamed('02 - Rental End Station Name','to_station_name')\
.withColumnRenamed('User Type','usertype')\
.withColumnRenamed('Member Gender','gender')\
.withColumnRenamed('05 - Member Details Member Birthday Year','birthyear')

d19.show(2),d19.count()

+--------+-------------------+-------------------+------+------------+---------------+-------------------+-------------+--------------------+----------+------+---------+
| trip_id|          starttime|           stoptime|bikeid|tripduration|from_station_id|  from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+--------+-------------------+-------------------+------+------------+---------------+-------------------+-------------+--------------------+----------+------+---------+
|22178529|2019-04-01 00:02:22|2019-04-01 00:09:48|  6251|       446.0|             81| Daley Center Plaza|           56|Desplaines St & K...|Subscriber|  Male|     1975|
|22178530|2019-04-01 00:03:02|2019-04-01 00:20:30|  6226|     1,048.0|            317|Wood St & Taylor St|           59|Wabash Ave & Roos...|Subscriber|Female|     1984|
+--------+-------------------+-------------------+------+------------+---------------+-------------------+-------------+--------------------+---------

(None, 3818004)

In [25]:
d20=spark.read.csv('gs://buck_561/divvy15_21/2020',header=True,inferSchema=True)
d20.show(2),d20.count()

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+----------+-------------+--------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|start_lat| start_lng|      end_lat|       end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+----------+-------------+--------------+-------------+
|762198876D69004D|  docked_bike|2020-07-09 15:22:02|2020-07-09 15:25:52|Ritchie Ct & Bank...|             180|Wells St & Evergr...|           291|41.906866|-87.626217|    41.906724|     -87.63483|       member|
|BEC9C9FBA0D4CF1B|  docked_bike|2020-07-24 23:56:30|2020-07-25 00:20:17|Halsted St & Rosc...|             299|Broadway & Ridge Ave|           461| 41.94367|

(None, 3541683)

In [26]:
d21=spark.read.csv('gs://buck_561/divvy15_21/2021',header=True,inferSchema=True)
d21.show(2),d21.count()

+----------------+-------------+-------------------+-------------------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|start_station_name|start_station_id|end_station_name|end_station_id|start_lat|start_lng|end_lat|end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+
|C809ED75D6160B2A|electric_bike|2021-05-30 11:58:15|2021-05-30 12:10:39|              null|            null|            null|          null|     41.9|   -87.63|  41.89| -87.61|       casual|
|DD59FDCE0ACACAF3|electric_bike|2021-05-30 11:29:14|2021-05-30 12:14:09|              null|            null|            null|          null|    41.88|   -87.62|  41.79| -87.58|       casual|
+----------------+-------------+-------------

(None, 5595063)

In [27]:
d22=spark.read.csv('gs://buck_561/divvy15_21/2022',header=True,inferSchema=True)
d22.show(2),d22.count()

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+----------+-----------+--------------+--------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id| start_lat|  start_lng|       end_lat|       end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+----------+-----------+--------------+--------------+-------------+
|C2F7DD78E82EC875|electric_bike|2022-01-13 11:59:47|2022-01-13 12:02:44|Glenwood Ave & To...|             525|Clark St & Touhy Ave|        RP-007|42.0128005| -87.665906|42.01256011541|-87.6743671152|       casual|
|A6CF8980A652D272|electric_bike|2022-01-10 08:41:56|2022-01-10 08:46:17|Glenwood Ave & To...|             525|Clark St & Touhy Ave|        RP-00

(None, 219379)

# formating

In [28]:
d1415=d14\
.union(d15.select(d14.columns))\
.withColumn('starttime',F.concat(F.col('starttime'),F.lit(':00')))\
.withColumn('stoptime',F.concat(F.col('stoptime'),F.lit(':00')))

d1617=d16.union(d17.select(d16.columns))
d14151617=d1415.union(d1617.select(d1415.columns))

df1=d14151617.withColumn('month', split(d14151617['starttime'], '/').getItem(0))\
.withColumn('day', split(d14151617['starttime'], '/').getItem(1))\
.withColumn('year', split(d14151617['starttime'], '/').getItem(2))\
.withColumn('time', split(d14151617['starttime'], ' ').getItem(1))

df1=df1.withColumn('hr', split(df1['time'], ':').getItem(0))\
.withColumn('mins', split(df1['time'], ':').getItem(1))\
.withColumn('secs', split(df1['time'], ':').getItem(2))

df1=df1.withColumn('year', split(df1['year'], ' ').getItem(0))

df1=df1.withColumn("month",df1.month.cast(IntegerType()))\
.withColumn("day",df1.day.cast(IntegerType()))\
.withColumn("hr",df1.hr.cast(IntegerType()))\
.withColumn("month", F.format_string("%02d", "month"))\
.withColumn("day", F.format_string("%02d", "day"))\
.withColumn("hr", F.format_string("%02d", "hr"))

df1=df1.withColumn('fulldate',F.concat_ws('-',df1.year,df1.month,df1.day))\
.withColumn('time',F.concat_ws(':',df1.hr,df1.mins,df1.secs))

df1=df1.withColumn('starttime',F.concat_ws(' ',df1.fulldate,df1.time)).drop('month','day','year','time','fulldate','hr','mins','secs')

df1=df1.withColumn('month', split(df1['stoptime'], '/').getItem(0))\
.withColumn('day', split(df1['stoptime'], '/').getItem(1))\
.withColumn('year', split(df1['stoptime'], '/').getItem(2))\
.withColumn('time', split(df1['stoptime'], ' ').getItem(1))

df1=df1.withColumn('hr', split(df1['time'], ':').getItem(0))\
.withColumn('mins', split(df1['time'], ':').getItem(1))\
.withColumn('secs', split(df1['time'], ':').getItem(2))

df1=df1.withColumn('year', split(df1['year'], ' ').getItem(0))

df1=df1.withColumn("month",df1.month.cast(IntegerType()))\
.withColumn("day",df1.day.cast(IntegerType()))\
.withColumn("hr",df1.hr.cast(IntegerType()))\
.withColumn("month", F.format_string("%02d", "month"))\
.withColumn("day", F.format_string("%02d", "day"))\
.withColumn("hr", F.format_string("%02d", "hr"))

df1=df1.withColumn('fulldate',F.concat_ws('-',df1.year,df1.month,df1.day))\
.withColumn('time',F.concat_ws(':',df1.hr,df1.mins,df1.secs))

d14151617=df1.withColumn('stoptime',F.concat_ws(' ',df1.fulldate,df1.time)).drop('month','day','year','time','fulldate','hr','mins','secs')

d14151617.show(2)

+-------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|trip_id|          starttime|           stoptime|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+-------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|4413167|2014-12-31 23:54:00|2014-12-31 23:57:00|  1880|         193|            296|Broadway & Belmon...|          334|Lake Shore Dr & B...|Subscriber|  Male|     1989|
|4413166|2014-12-31 23:50:00|2014-12-31 23:52:00|  1153|         161|            264|Stetson Ave & Sou...|           44|State St & Randol...|Subscriber|  Male|     1984|
+-------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+---------

In [29]:
d131819=d13.union(d18.select(d13.columns)).union(d19.select(d13.columns))

d202122=d20\
.union(d21.select(d20.columns))\
.union(d22.select(d20.columns))\
.withColumnRenamed('started_at','starttime')\
.withColumnRenamed('ended_at','stoptime')\
.withColumnRenamed('start_station_name','from_station_name')\
.withColumnRenamed('start_station_id','from_station_id')\
.withColumnRenamed('end_station_id','to_station_id')\
.withColumnRenamed('end_station_name','to_station_name')\
.withColumnRenamed('member_casual','usertype')\
.withColumnRenamed('ride_id','trip_id')\
.withColumn('tripduration',F.lit('null'))\
.withColumn('birthyear',F.lit('null'))\
.withColumn('gender',F.lit('null'))\
.withColumn('bikeid',F.lit('null')).select(d18.columns)

d131819202122=d202122.union(d131819.select(d202122.columns))

df_final=d14151617.union(d131819202122.select(d14151617.columns))

In [30]:
df_2 = df_final.withColumn("starttime",F.date_format("starttime","yyyy-MM-dd hh:mm:ss"))\
.withColumn("starttime",F.to_timestamp("starttime","yyyy-MM-dd hh:mm:ss"))\
.withColumn("stoptime",F.date_format("stoptime","yyyy-MM-dd hh:mm:ss"))\
.withColumn("stoptime",F.to_timestamp("stoptime","yyyy-MM-dd hh:mm:ss"))

df_2.show(5),df_2.printSchema()

+-------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|trip_id|          starttime|           stoptime|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+-------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|4413167|2014-12-31 11:54:00|2014-12-31 11:57:00|  1880|         193|            296|Broadway & Belmon...|          334|Lake Shore Dr & B...|Subscriber|  Male|     1989|
|4413166|2014-12-31 11:50:00|2014-12-31 11:52:00|  1153|         161|            264|Stetson Ave & Sou...|           44|State St & Randol...|Subscriber|  Male|     1984|
|4413165|2014-12-31 11:48:00|2015-01-01 00:38:00|  1946|        3002|            168|Michigan Ave & 14...|          168|Michigan Ave & 14...|  Custome

(None, None)

In [31]:
df_2.write.mode('overwrite').parquet('gs://buck_561/divvy15_21/all.parquet')

In [32]:
df=spark.read.parquet('gs://buck_561/divvy15_21/all.parquet')

In [33]:
commaToDot = udf(lambda x : float(str(x).replace(',', '')), FloatType())

df = df.withColumn("dur", F.when(F.col("tripduration")=="null", F.unix_timestamp('stoptime')- F.unix_timestamp('starttime')).otherwise(F.col("tripduration")))
df1=df.withColumn('tripduration',commaToDot(df['dur'])).drop('dur')

df1.show()

+----------------+-------------------+-------------------+------+------------+---------------+-----------------+-------------+--------------------+--------+------+---------+
|         trip_id|          starttime|           stoptime|bikeid|tripduration|from_station_id|from_station_name|to_station_id|     to_station_name|usertype|gender|birthyear|
+----------------+-------------------+-------------------+------+------------+---------------+-----------------+-------------+--------------------+--------+------+---------+
|99FEC93BA843FB20|2021-06-13 02:31:28|2021-06-13 02:34:11|  null|       163.0|           null|             null|         null|                null|  member|  null|     null|
|06048DCFC8520CAF|2021-06-04 11:18:02|2021-06-04 11:24:19|  null|       377.0|           null|             null|         null|                null|  member|  null|     null|
|9598066F68045DF2|2021-06-04 09:49:35|2021-06-04 09:55:34|  null|       359.0|           null|             null|         null|    

In [34]:
df1.describe().show()

+-------+----------------+------------------+------------------+--------------------+-------------------+--------------------+-------------------+--------+--------+------------------+
|summary|         trip_id|            bikeid|      tripduration|     from_station_id|  from_station_name|       to_station_id|    to_station_name|usertype|  gender|         birthyear|
+-------+----------------+------------------+------------------+--------------------+-------------------+--------------------+-------------------+--------+--------+------------------+
|  count|        30599469|          30599469|          30599469|            29778541|           29779164|            29710675|           29711136|30599469|25704509|          25732676|
|   mean|        Infinity|2900.2971443196516|1055.9930605985353|    1243.99852550927|              351.0|  1236.2722204621764|               null|    null|    null|1981.4292950328797|
| stddev|             NaN| 1813.878139789572| 19571.77381836103|  3801.909314610

In [35]:
df1.filter('tripduration != "null"').describe().show()


+-------+-------+------+------------+---------------+-----------------+-------------+---------------+--------+------+---------+
|summary|trip_id|bikeid|tripduration|from_station_id|from_station_name|to_station_id|to_station_name|usertype|gender|birthyear|
+-------+-------+------+------------+---------------+-----------------+-------------+---------------+--------+------+---------+
|  count|      0|     0|           0|              0|                0|            0|              0|       0|     0|        0|
|   mean|   null|  null|        null|           null|             null|         null|           null|    null|  null|     null|
| stddev|   null|  null|        null|           null|             null|         null|           null|    null|  null|     null|
|    min|   null|  null|        null|           null|             null|         null|           null|    null|  null|     null|
|    max|   null|  null|        null|           null|             null|         null|           null|   

In [36]:
df1.filter('tripduration = "null"').describe().show()


+-------+-------+------+------------+---------------+-----------------+-------------+---------------+--------+------+---------+
|summary|trip_id|bikeid|tripduration|from_station_id|from_station_name|to_station_id|to_station_name|usertype|gender|birthyear|
+-------+-------+------+------------+---------------+-----------------+-------------+---------------+--------+------+---------+
|  count|      0|     0|           0|              0|                0|            0|              0|       0|     0|        0|
|   mean|   null|  null|        null|           null|             null|         null|           null|    null|  null|     null|
| stddev|   null|  null|        null|           null|             null|         null|           null|    null|  null|     null|
|    min|   null|  null|        null|           null|             null|         null|           null|    null|  null|     null|
|    max|   null|  null|        null|           null|             null|         null|           null|   

In [40]:
df1.filter('tripduration = -1770041.0').show()

+----------------+-------------------+-------------------+------+------------+---------------+-----------------+-------------+--------------------+--------+------+---------+
|         trip_id|          starttime|           stoptime|bikeid|tripduration|from_station_id|from_station_name|to_station_id|     to_station_name|usertype|gender|birthyear|
+----------------+-------------------+-------------------+------+------------+---------------+-----------------+-------------+--------------------+--------+------+---------+
|23AB8FC1763631EB|2020-12-15 11:42:13|2020-11-25 00:01:32|  null|  -1770041.0|   TA1307000039|Clark St & Elm St|        13059|Bissell St & Armi...|  member|  null|     null|
+----------------+-------------------+-------------------+------+------------+---------------+-----------------+-------------+--------------------+--------+------+---------+



In [39]:
df1.filter('tripduration > 3700 ').count()

1028986

In [41]:
df.filter('trip_id = "23AB8FC1763631EB"').show()

+----------------+-------------------+-------------------+------+------------+---------------+-----------------+-------------+--------------------+--------+------+---------+--------+
|         trip_id|          starttime|           stoptime|bikeid|tripduration|from_station_id|from_station_name|to_station_id|     to_station_name|usertype|gender|birthyear|     dur|
+----------------+-------------------+-------------------+------+------------+---------------+-----------------+-------------+--------------------+--------+------+---------+--------+
|23AB8FC1763631EB|2020-12-15 11:42:13|2020-11-25 00:01:32|  null|        null|   TA1307000039|Clark St & Elm St|        13059|Bissell St & Armi...|  member|  null|     null|-1770041|
+----------------+-------------------+-------------------+------+------------+---------------+-----------------+-------------+--------------------+--------+------+---------+--------+



In [43]:
df1.filter('tripduration < 0 ').show()

+----------------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+--------+------+---------+
|         trip_id|          starttime|           stoptime|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|usertype|gender|birthyear|
+----------------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+--------+------+---------+
|A714D49E229150CD|2021-06-04 11:51:48|2021-06-04 00:42:02|  null|    -40186.0|           null|                null|         null|                null|  member|  null|     null|
|BD20ACF1862503B5|2021-06-24 11:26:21|2021-06-24 00:12:08|  null|    -40453.0|           null|                null|         null|                null|  casual|  null|     null|
|0D24A048CDA47E50|2021-06-24 11:26:36|2021-06-24 00:11:51|  null|    -40485.0|           null|                null|

In [44]:
df1.write.mode('overwrite').parquet('gs://buck_561/divvy15_21/all1.parquet')
